## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-10-13-16-50-25 +0000,nyt,"Small Plane Crashes on Massachusetts Highway, ...",https://www.nytimes.com/2025/10/13/us/highway-...
1,2025-10-13-16-48-51 +0000,nyt,Live Updates: Hostages and Palestinian Prisone...,https://www.nytimes.com/live/2025/10/12/world/...
2,2025-10-13-16-48-39 +0000,nyt,Here’s the latest.,https://www.nytimes.com/live/2025/10/12/world/...
3,2025-10-13-16-48-00 +0000,wsj,"Joel Mokyr, Philippe Aghion and Peter Howitt w...",https://www.wsj.com/economy/nobel-economics-pr...
4,2025-10-13-16-46-27 +0000,startribune,Minneapolis Downtown Council pulls plug on Hol...,https://www.startribune.com/downtown-minneapol...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2332/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
12,trump,49
151,china,22
62,israel,18
8,hostages,18
403,hamas,13


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
6,2025-10-13-16-42-31 +0000,wapo,"Live updates: Trump says war in Gaza ‘is over,...",https://www.washingtonpost.com/world/2025/10/1...,129
229,2025-10-12-22-52-05 +0000,nypost,Hillary Clinton heaps rare praise on President...,https://nypost.com/2025/10/12/us-news/hillary-...,119
85,2025-10-13-13-11-27 +0000,nypost,"Dow soars nearly 600 points after Trump, Vance...",https://nypost.com/2025/10/13/business/dow-fut...,107
185,2025-10-13-05-31-15 +0000,nypost,‘War is over’ Trump declares — as he arrives i...,https://nypost.com/2025/10/13/world-news/war-i...,104
175,2025-10-13-07-20-04 +0000,nypost,"What Trump told Netanyahu, father of ambassado...",https://nypost.com/2025/10/13/world-news/trump...,98


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
6,129,2025-10-13-16-42-31 +0000,wapo,"Live updates: Trump says war in Gaza ‘is over,...",https://www.washingtonpost.com/world/2025/10/1...
264,56,2025-10-12-17-31-50 +0000,nypost,Father of Israeli-American hostage assumed mur...,https://nypost.com/2025/10/12/world-news/hosta...
3,51,2025-10-13-16-48-00 +0000,wsj,"Joel Mokyr, Philippe Aghion and Peter Howitt w...",https://www.wsj.com/economy/nobel-economics-pr...
9,50,2025-10-13-16-39-58 +0000,nypost,"FIFA president, over two dozen world leaders a...",https://nypost.com/2025/10/13/us-news/fifa-pre...
56,49,2025-10-13-14-55-52 +0000,nypost,JPMorgan to invest $10B in areas ‘critical’ to...,https://nypost.com/2025/10/13/business/jpmorga...
128,32,2025-10-13-10-31-45 +0000,nypost,"Israeli twin brothers Ziv, Gali Berman reunite...",https://nypost.com/2025/10/13/world-news/israe...
31,31,2025-10-13-16-00-00 +0000,wsj,The relationship between capitalism and democr...,https://www.wsj.com/politics/united-states-ame...
181,31,2025-10-13-06-16-39 +0000,nypost,Last living hostages all released after 738 da...,https://nypost.com/2025/10/13/world-news/last-...
228,30,2025-10-12-23-00-00 +0000,wsj,Federal workers are already starting to miss p...,https://www.wsj.com/politics/policy/the-govern...
180,28,2025-10-13-06-21-00 +0000,bbc,"Boy, 11, shoots farm worker dead in tragic Sou...",https://www.bbc.com/news/articles/ce86dvr03zqo...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
